In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [40]:
def card_info(url):
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    #Headers for the table
    card_info_headers = soup.find_all(attrs = {'class':'cardtablerowheader'})
    headers = []
    for header in card_info_headers:
        headers.append(header.text)
    headers = headers[:-2]
    
    #Information for the table
    card_info_table = soup.find_all(attrs = {'class':'cardtablerowdata'})
    info = []
    for table_entry in card_info_table:
        info.append(table_entry.text)
    info = info[:-2]
    info_2 = []
    for clown in info:
        if '\n' in clown:
            clown_2 = clown.replace('\n','')
            info_2.append(clown_2)
        else:
            info_2.append(clown)
    #Removing the names in different languages
    while not ( info_2[1] == 'Spell ' or info_2[1] == 'Monster ' or info_2[1] == ('Trap ')):
        info_2.remove(info_2[1])
        headers.remove(headers[1])
    else:
        info_2 = info_2
    if ('Traditional' in info_2[-1]) == True:
        if ('TCG Advanced') in info_2[-1]:
            headers.remove(headers[-2])
            info_2.remove(info_2[-2])
        else:
            info_2.remove(info_2[-1])
            info_2.remove(info_2[-2])
            headers.remove(headers[-1])
            headers.remove(headers[-2])
    #Adding the effect
    Jeff = soup.find(attrs = {'class':'collapsible expanded navbox-inner'})
    Jeff = Jeff.text[12:-2]
    info_2.append(Jeff)
    headers.append('Card Text')
    info_dict = {headers[i] : info_2[i] for i in range(0, len(headers))}
    return info_dict

In [60]:
class TCG_set:
    
    def __init__(self, url):
        self.url = url
        result = requests.get(url)
        src = result.content
        soup = BeautifulSoup(src, 'html.parser')
        #Getting the name of the set
        headers_soup = soup.find_all("h2")
        self.set_name = headers_soup[0].text
        #Getting initial set info
        Destiny_1 = soup.find_all('section')
        section_headers = []
        section_headers_txt = []
        for Aster in Destiny_1:
            a = Aster.find_all('h2')
            section_headers.append(a) 
        for header in section_headers:
            if header != []:
                for head in header:
                    section_headers_txt.append(head.text)
            else:
                    section_headers_txt.append('lol')
        #Index of Set Info and Release Date
        ind_1 = section_headers_txt.index('Set Information')
        #Adding dict keys/values to an empty list here
        sub_headers = []
        more_set_info = []
        #Adding set info to the dictionary
        Destiny_2 = Destiny_1[ind_1].find_all(attrs = {'class' : "pi-data-value pi-font"})
        Destiny_4 = Destiny_1[ind_1].find_all(attrs = {'class' : "pi-data-label pi-secondary-font"})
        for Aster in Destiny_2:
            more_set_info.append(Aster.text)
        for Aster in Destiny_4:
            sub_headers.append(Aster.text)
        if 'Release dates' in section_headers_txt:
            ind_3 = section_headers_txt.index('Release dates')
            #Adding release dates to the dictionary
            Destiny_3 = Destiny_1[ind_3].find_all(attrs = {'class' : "pi-data-value pi-font"})
            Destiny_5 = Destiny_1[ind_3].find_all(attrs = {'class' : "pi-data-label pi-secondary-font"})
            for Aster in Destiny_3:
                more_set_info.append(Aster.text)
            for Aster in Destiny_5:
                sub_headers.append(Aster.text + ' Release Date')
        #Making the Dictionary
        sub_head_dict = {sub_headers[i] : more_set_info[i] for i in range(0, len(sub_headers))}
        if 'Prefix(es)' in list(sub_head_dict.keys()):
            set_code = sub_head_dict['Prefix(es)']
            if '-ENSE' in set_code:
                a = set_code.find('-')
                sub_head_dict['Prefix(es)']  = set_code[:a+5]
            else:    
                a = set_code.find('-')
                sub_head_dict['Prefix(es)'] = set_code[:a]
        if ' \(en' in sub_head_dict['Prefix(es)']:
            b = sub_head_dict['Prefix(es)'].find(' ')
            sub_head_dict['Prefix(es)'] = sub_head_dict['Prefix(es)'][:b]
        sub_head_dict['Name'] = headers_soup[0].text
        sub_head_dict['Wiki link'] = url
        self.set_info = sub_head_dict
        #Contents of the set
        Destiny = soup.find_all("table")
        #Get the headers for each table so that we can find each table that we actually want
        tables = []
        for Aster in Destiny:
            bruh = []
            table_data = Aster
            table_headers = table_data.find_all("th")
            for jeff in table_headers:
                bruh.append(jeff.text)
            tables.append(bruh)
        #Having found the table, putting it all into a dataframe. We use concat to make sure stuff like the Legendary Heroes 
        #decks get all their cards put in the set  
        dataframes = []
        for j in range(0, len(tables)):
            if (('Card number' in tables[j]) and ('Name' in tables[j])) or (('Set number\n' in tables[j]) and ('Name\n' in tables[j])) or (tables[j] == ['Set number\n', 'English name\n', 'Rarity\n', 'Category\n']):
                #a = tables.index(table) #This line is the problem
                table_entries = Destiny[j].find_all('td')
                #Priming the datasets now, depending on if there's a "quantity" column or not
                if (len(tables[j]) == 5):
                    card_number= []
                    name= []
                    rarity= []
                    category = []
                    quantity = []
                    for i in range(0, len(table_entries)):
                        if (i % 5 == 0):
                            card_number.append(table_entries[i].text)
                        elif (i % 5 == 1):
                            name.append(table_entries[i].text)
                        elif (i % 5 == 2):
                            rarity.append(table_entries[i].text)
                        elif (i % 5 == 3):
                            category.append(table_entries[i].text)
                        else:
                            quantity.append(table_entries[i].text)
                    set_content = pd.DataFrame({'Card number': card_number, 'Name': name, 
                                                     'Rarity': rarity ,'Category': category, 'Quantity': quantity})
                    dataframes.append(set_content)
                if (len(tables[j]) == 4):
                    card_number= []
                    name= []
                    rarity= []
                    category = []
                    for i in range(0, len(table_entries)):
                        if (i % 4 == 0):
                            card_number.append(table_entries[i].text)
                        elif (i % 4 == 1):
                            name.append(table_entries[i].text)
                        elif (i % 4 == 2):
                            rarity.append(table_entries[i].text)
                        else:
                            category.append(table_entries[i].text)
                    set_content = pd.DataFrame({'Card number': card_number, 'Name': name, 
                                                     'Rarity': rarity ,'Category': category})
                    dataframes.append(set_content)
                    
        set_content = pd.concat(dataframes, ignore_index = True)
        names = []
        urls = []
        for name in set_content['Name']:
            if (name[0] == '\"') and (name[-1] == '\"'):
                name = name[1:-1]
            name = name.replace(' ', '_')
            url = 'https://yugioh.fandom.com/wiki/' + name
            names.append(name)
            urls.append(url)
        set_content['Name'] = names
        set_content['Card urls'] = urls
        card_infos = []
        for url in list(set_content['Card urls']):
            card_infos.append(card_info(url))
        df_all_cards = pd.DataFrame(card_infos)
        df_xtd = pd.concat([set_content, df_all_cards], axis = 1)
        
        self.set_content = df_xtd
        #Rarity breakdown
        self.rarity_breakdown = dict(self.set_content['Rarity'].value_counts())